In [ ]:
!pip install faiss-cpu bitsandbytes

In [ ]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import shutil
import os
from tqdm import tqdm
from peft import PeftModel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
source_folder = '/content/drive/MyDrive/NLP_Paper_Review/dataset'
destination_folder = '/content/dataset'

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Copy the folder and its contents
shutil.copytree(source_folder, destination_folder, dirs_exist_ok=True)

print(f"Files copied from {source_folder} to {destination_folder}")

source_path = '/content/drive/MyDrive/NLP_Paper_Review/ft-mistral.zip'
destination_path = '/content/ft-mistral.zip'

# Copy the file
shutil.copy(source_path, destination_path)
print(f"Model copied from {source_path} to {destination_path}")

# Extract the model
import zipfile
with zipfile.ZipFile(destination_path, 'r') as zip_ref:
    zip_ref.extractall('extracted_model')

Files copied from /content/drive/MyDrive/NLP_Paper_Review/dataset to /content/dataset
Model copied from /content/drive/MyDrive/NLP_Paper_Review/ft-mistral.zip to /content/ft-mistral.zip


In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')
# Load the model from the extracted directory

BASE_ID = "mistralai/Mistral-7B-Instruct-v0.2"        # original weights

bnb_conf = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,            # change to float16 if bf16 unsupported
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

base = AutoModelForCausalLM.from_pretrained(
        "mistralai/Mistral-7B-Instruct-v0.2",
        quantization_config=bnb_conf,
        torch_dtype=torch.bfloat16,
        device_map={"": 0},
        trust_remote_code=True,
)

# ❷  attach your LoRA deltas
model = PeftModel.from_pretrained(base, "/content/extracted_model/ft-mistral")
tokenizer = AutoTokenizer.from_pretrained(BASE_ID, use_fast=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def predict(question, options):

    # Format options
    options_str = "\n".join([f"{option}" for ind, option in enumerate(options)])

    instructions = "Just provide the answer to the Question at the end of the prompt based on the sample context questions provided. Mention the option number ONLY. Use Numbers for options, not anything else."

    instructions = ""

    augmented_prompt = f"{instructions}\n\n{question}\nOptions:\n{options_str}\n\nAnswer:"

    # Generate response - move inputs to the same device as model
    inputs = tokenizer(augmented_prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Move inputs to model's device

    # Generate only new tokens (the answer)
    prompt_length = inputs["input_ids"].shape[1]
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    # Extract only the newly generated tokens (the answer part)
    answer_tokens = outputs[0][prompt_length:]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
    # return answer.strip()
    answer_option = answer.strip().split('\n')[0]
    return answer_option

In [ ]:
query = "How have we been able to construct detailed maps of surface features on Venus?"
options = ["by studying Venus from Earth with powerful optical telescopes","by landing spacecraft on the surface for close-up study","by studying Venus with powerful optical telescopes on spacecraft that were sent to orbit Venus","by using radar from spacecraft that were sent to orbit Venus"]
response = predict(query, options)
print(response)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


by using radar from spacecraft that were sent to orbit Venus


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
st_model = SentenceTransformer('all-MiniLM-L6-v2')
def get_pred_indexes(pred, dataset) :
  pred_indexes = []
  for ind, pred in enumerate(pred) :
    choices = dataset[ind]['choices']
    pred_embedding = st_model.encode([pred], normalize_embeddings=True)
    choice_embeddings = st_model.encode(choices, normalize_embeddings=True)
    similarities = cosine_similarity(pred_embedding, choice_embeddings)[0]
    # Find the most similar sentence
    most_similar_idx = int(np.argmax(similarities))
    pred_indexes.append(most_similar_idx)
  return pred_indexes

In [ ]:
validation = json.load(open('dataset/mmlu_datasets/mmlu_astronomy_validation.json'))
test = json.load(open('dataset/mmlu_datasets/mmlu_astronomy_test.json'))

validation[0]

{'question': 'You cool a blackbody to half its original temperature. How does its spectrum change?',
 'subject': 'astronomy',
 'choices': ['Power emitted is 1/16 times as high; peak emission wavelength is 1/2 as long.',
  'Power emitted is 1/4 times as high; peak emission wavelength is 2 times longer.',
  'Power emitted is 1/4 times as high; peak emission wavelength is 1/2 as long.',
  'Power emitted is 1/16 times as high; peak emission wavelength is 2 times longer.'],
 'answer': 3}

In [ ]:
val_pred = []
val_actual = [item['answer'] for item in validation]
for item in tqdm(validation, desc='Validating'):
  pred = predict(item['question'], item['choices'])
  val_pred.append(pred)

Validating:   0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Validating: 100%|██████████| 16/16 [01:06<00:00,  4.13s/it]


In [ ]:
val_pred[:5], val_actual[:5]

(['Power emitted is 1/4 times as high; peak emission wavelength is 1/2 as long.',
  'The minimization of gravitational potential energy.',
  "Large impacts fractured the Moon's lithosphere allowing lava to fill the impact basins.",
  'It does not rotate fast enough.',
  '5 : 1'],
 [3, 1, 3, 0, 2])

In [ ]:
val_pred_indexes = get_pred_indexes(val_pred, validation)
val_pred_indexes, val_actual

([2, 1, 3, 2, 2, 2, 3, 2, 2, 1, 1, 0, 2, 1, 3, 1],
 [3, 1, 3, 0, 2, 1, 3, 3, 0, 0, 1, 0, 2, 1, 3, 0])

In [ ]:
accuracy = (np.array(val_pred_indexes) == np.array(val_actual)).mean()
accuracy

np.float64(0.5625)

In [ ]:
import pandas as pd

data = []
for ind, item in enumerate(validation):
    # Extract the question, choices, and answer index
    question = item['question']
    choices = item['choices']
    answer_idx = item['answer']

    # Create a dictionary for this row
    row = {
        'question': question,
        'option_0': choices[0],
        'option_1': choices[1],
        'option_2': choices[2],
        'option_3': choices[3],
        'answer_idx': answer_idx,
        'predicted_idx': val_pred_indexes[ind]
    }

    # Add to our data list
    data.append(row)
df = pd.DataFrame(data)
df.head()
df.to_csv('ft_validation_results.csv', index=False)

In [ ]:
test_pred = []
test_actual = [item['answer'] for item in test]
for item in tqdm(test, desc="Testing"):
  pred = predict(item['question'], item['choices'])
  test_pred.append(pred)

Testing:   0%|          | 0/152 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Testing: 100%|██████████| 152/152 [11:16<00:00,  4.45s/it]


In [ ]:
test_pred_indexes = get_pred_indexes(test_pred, test)
test_pred_indexes[0:5], test_actual[0:5]

([0, 3, 2, 3, 3], [0, 3, 2, 2, 3])

In [ ]:
accuracy = (np.array(test_pred_indexes) == np.array(test_actual)).mean()
accuracy

np.float64(0.6842105263157895)

In [ ]:
data = []
for ind, item in enumerate(test):
    # Extract the question, choices, and answer index
    question = item['question']
    choices = item['choices']
    answer_idx = item['answer']

    # Create a dictionary for this row
    row = {
        'question': question,
        'option_0': choices[0],
        'option_1': choices[1],
        'option_2': choices[2],
        'option_3': choices[3],
        'answer_idx': answer_idx,
        'predicted_idx': test_pred_indexes[ind]
    }

    # Add to our data list
    data.append(row)
df = pd.DataFrame(data)
df.head()
df.to_csv('ft_test_results.csv', index=False)

# RAG + FT

In [ ]:
data = []
with open('dataset/training_data.json', 'r') as f:
  train_data: list[dict] = json.load(f)

# with open('dataset/astronomy_augmented_1.json', 'r') as f:
#   augmented_data_1: list[dict] = json.load(f)

with open('dataset/astronomy_augmented_2.json', 'r', encoding='UTF-8') as f:
  augmented_data_2: list[dict] = json.load(f)

data += train_data + augmented_data_2 # + augmented_data_1
len(data)

3472

In [ ]:
chunks = []
for item in data :
  question = item['question']
  choices = item['choices']
  answer_idx = item['answer']
  answer = choices[answer_idx]

  chunk = f'Question: {question} \nAnswer: {answer}'
  chunks.append(chunk)

chunks[0:5]

['Question: According to the article, what is an asteroid? \nAnswer: An object larger than a meteoroid that is neither a planet nor an identified comet, orbiting within the inner Solar System or co-orbital with Jupiter',
 'Question: What are the broad classifications of asteroids based on composition? \nAnswer: Carbonaceous, Metallic, Silicaceous',
 'Question: Where are the greatest number of known asteroids located? \nAnswer: Between the orbits of Mars and Jupiter',
 'Question: Which of the following spacecraft directly studied Vesta and Ceres? \nAnswer: Dawn',
 'Question: What potentially catastrophic event is associated with Near-Earth asteroids? \nAnswer: Colliding with Earth and causing mass extinction events']

In [ ]:
embedding_model = SentenceTransformer("BAAI/bge-large-en")
chunk_embeddings = embedding_model.encode(chunks, normalize_embeddings=True)

In [ ]:
dimension = chunk_embeddings.shape[1]
dimension

1024

In [ ]:
index = faiss.IndexFlatIP(dimension)
index.add(chunk_embeddings)

In [ ]:
def rag(question, options, k=3):
    # Generate query embedding and normalize
    question_embedding = embedding_model.encode([question], normalize_embeddings=True)

    # Retrieve top-k similar chunks
    scores, indices = index.search(question_embedding, k)

    # Construct context from retrieved chunks
    context = "\n\n".join([chunks[idx] for idx in indices[0]])

    # Format options
    options_str = "\n".join([f"{option}" for ind, option in enumerate(options)])

    instructions = "Just provide the answer to the Question at the end of the prompt based on the sample context questions provided. "

    # Create prompt
    augmented_prompt = f"{instructions}\n\n-----Beginning of Context------:\n{context}\n------End of Context-------\n\n\nQuestion to answer: {question}\nOptions to choose from:\n{options_str}\n\nAnswer:"

    # Generate response - move inputs to the same device as model
    inputs = tokenizer(augmented_prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Move inputs to model's device

    # Generate only new tokens (the answer)
    prompt_length = inputs["input_ids"].shape[1]
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    # Extract only the newly generated tokens (the answer part)
    answer_tokens = outputs[0][prompt_length:]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
    answer_option = answer.strip().split('\n')[0]
    return answer_option

In [ ]:
query = "How have we been able to construct detailed maps of surface features on Venus?"
options = ["by studying Venus from Earth with powerful optical telescopes","by landing spacecraft on the surface for close-up study","by studying Venus with powerful optical telescopes on spacecraft that were sent to orbit Venus","by using radar from spacecraft that were sent to orbit Venus"]
response = rag(query, options)
print(response)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


by using radar from spacecraft that were sent to orbit Venus


In [ ]:
from tqdm import tqdm

val_pred = []
val_actual = [item['answer'] for item in validation]

# Create progress bar
for item in tqdm(validation, desc="Validating"):
    pred = rag(item['question'], item['choices'])
    val_pred.append(pred)

Validating: 100%|██████████| 16/16 [01:33<00:00,  5.85s/it]


In [ ]:
val_pred[0:5]

['Power emitted is 1/16 times as high; peak emission wavelength is 2 times longer.',
 'The minimization of gravitational potential energy.',
 "Large impacts fractured the Moon's lithosphere allowing lava to fill the impact basins.",
 'Its rotation axis is nearly perpendicular to the plane of the Solar System.',
 '5 : 1']

In [ ]:
val_pred_indexes = get_pred_indexes(val_pred, validation)
val_pred_indexes, val_actual

([3, 1, 3, 0, 2, 1, 3, 3, 0, 0, 1, 0, 2, 1, 3, 0],
 [3, 1, 3, 0, 2, 1, 3, 3, 0, 0, 1, 0, 2, 1, 3, 0])

In [ ]:
accuracy = (np.array(val_pred_indexes) == np.array(val_actual)).mean()
accuracy

np.float64(1.0)

In [ ]:
data = []
for ind, item in enumerate(validation):
    # Extract the question, choices, and answer index
    question = item['question']
    choices = item['choices']
    answer_idx = item['answer']

    # Create a dictionary for this row
    row = {
        'question': question,
        'option_0': choices[0],
        'option_1': choices[1],
        'option_2': choices[2],
        'option_3': choices[3],
        'answer_idx': answer_idx,
        'predicted_idx': val_pred_indexes[ind]
    }

    # Add to our data list
    data.append(row)
df = pd.DataFrame(data)
df.head()
df.to_csv('ft_rag_validation_results.csv', index=False)

In [ ]:
test_pred = []
test_actual = [item['answer'] for item in test]
for item in tqdm(test, desc="Testing"):
  pred = rag(item['question'], item['choices'])
  test_pred.append(pred)

Testing:   0%|          | 0/152 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Testing: 100%|██████████| 152/152 [14:16<00:00,  5.64s/it]


In [ ]:
test_pred_indexes = get_pred_indexes(test_pred, test)
test_pred_indexes[0:15], test_actual[0:15]

([0, 3, 2, 3, 3, 3, 3, 1, 3, 2, 3, 2, 0, 3, 1],
 [0, 3, 2, 2, 3, 2, 1, 1, 3, 3, 3, 2, 0, 3, 1])

In [ ]:
accuracy = (np.array(test_pred_indexes) == np.array(test_actual)).mean()
accuracy

np.float64(0.631578947368421)

In [ ]:
data = []
for ind, item in enumerate(test):
    # Extract the question, choices, and answer index
    question = item['question']
    choices = item['choices']
    answer_idx = item['answer']

    # Create a dictionary for this row
    row = {
        'question': question,
        'option_0': choices[0],
        'option_1': choices[1],
        'option_2': choices[2],
        'option_3': choices[3],
        'answer_idx': answer_idx,
        'predicted_idx': test_pred_indexes[ind]
    }

    # Add to our data list
    data.append(row)
df = pd.DataFrame(data)
df.head()
df.to_csv('ft_rag_test_results.csv', index=False)